In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/datasets/distilbert_results

/content/drive/MyDrive/datasets/distilbert_results


In [18]:
!pip install keras-tuner --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.5 MB/s eta 0:00:00


In [19]:
import pandas as pd
import numpy as np
import tensorflow as tf
import kerastuner as kt

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Bidirectional
from tensorflow.keras.optimizers import Adam

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

<ipython-input-19-3fb584ba949c>:4: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [4]:
# Check TensorFlow version and GPU availability
print("TensorFlow version:", tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Get GPU details if available
if tf.config.list_physical_devices('GPU'):
    gpu_info = tf.config.list_physical_devices('GPU')[0]
    print("GPU Name:", gpu_info.name)

TensorFlow version: 2.15.0
Num GPUs Available:  1
GPU Name: /physical_device:GPU:0


In [5]:
ingredients = pd.read_csv('/content/drive/MyDrive/datasets/distilbert_results/ner_ingredients.csv')
labels = pd.read_csv('/content/drive/MyDrive/datasets/distilbert_results/ner_labels.csv')
recipes = pd.read_csv('/content/drive/MyDrive/datasets/distilbert_results/ner_recipes.csv')

In [12]:
ingredients.head()

,ingredients
0,mango juice
1,pinch salt
2,rout
3,rill grate
4,dill gherkin


In [13]:
labels.head()

,Vegetarian,Vegetarian&Desserts
0,0,1
1,0,0
2,0,0
3,1,1
4,0,1


In [14]:
recipes.head()

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,42
0,banana,sugar,cinnamon,vanilla ice cream,bourbon,butter,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,mash potato,milk,tomato,beef broth,rosemary leave,skirt steak,pepper,butter,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,clove,jalapeno,ginger,bourbon,apple cider,lemon,anise,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,soy sauce,sugar,sesame oil,asparagus,sesame seed,wine vinegar,kosher salt,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,egg,baking soda,sugar,flour,cream,confectioner,cream cheese,beer,stick butter,cupcake,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
recipes_lst = [ [item for item in row if not pd.isna(item)] for row in recipes.values ]

In [26]:
recipes_lst[0]

['banana', 'sugar', 'cinnamon', 'vanilla ice cream', 'bourbon', 'butter']

In [7]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(recipes_lst)  # Fit the tokenizer on the texts

sequences = tokenizer.texts_to_sequences(recipes_lst)

In [34]:
vocab_size = len(tokenizer.word_index) + 1

In [9]:
max_length = max(len(x) for x in sequences)
recipes_sequences = pad_sequences(sequences, maxlen=max_length, padding='post', dtype='float32', value=0.0)  # Post-padding with zeros

In [10]:
recipes_sequences.shape

(9770, 43)

In [11]:
labels_lst = labels['Vegetarian&Desserts'].values

In [12]:
type(labels_lst)

numpy.ndarray

In [13]:
X_train, X_test, y_train, y_test = train_test_split(recipes_sequences, labels_lst, test_size=0.3, random_state=42, stratify=labels)

In [67]:
# Define model
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=100, input_length=43),
    Bidirectional(LSTM(256)),
    Dense(1, activation='sigmoid')
])

# Compile model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Summary of the model
model.summary()

# Assuming X_train and y_train are your data prepared as sequences and labels respectively
# Train the model
model.fit(X_train, y_train, batch_size=32, epochs=10, validation_split=0.2)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 43, 100)           598300    
                                                                 
 bidirectional_3 (Bidirecti  (None, 512)               731136    
 onal)                                                           
                                                                 
 dense_3 (Dense)             (None, 1)                 513       
                                                                 
Total params: 1329949 (5.07 MB)
Trainable params: 1329949 (5.07 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
171/171 [==============================] - 84s 461ms/step - loss: 0.5607 - accuracy: 0.7128 - val_loss: 0.4622 - val_accuracy: 0.7975
Epoch 2/10
171/171 [==============================] - 78s 459ms/step 

In [53]:
# Define model
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=100, input_length=43),
    Bidirectional(LSTM(1)),
    Dense(1, activation='sigmoid')
])

# Compile model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Summary of the model
model.summary()

# Assuming X_train and y_train are your data prepared as sequences and labels respectively
# Train the model
model.fit(X_train, y_train, batch_size=32, epochs=10, validation_split=0.2)

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_14 (Embedding)    (None, 43, 100)           598300    
                                                                 
 bidirectional_14 (Bidirect  (None, 2)                 816       
 ional)                                                          
                                                                 
 dense_14 (Dense)            (None, 1)                 3         
                                                                 
Total params: 599119 (2.29 MB)
Trainable params: 599119 (2.29 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
171/171 [==============================] - 16s 52ms/step - loss: 0.6581 - accuracy: 0.6341 - val_loss: 0.5584 - val_accuracy: 0.7449
Epoch 2/10
171/171 [==============================] - 4s 23ms/step - lo

In [69]:
# Assuming X_test is prepared and ready
predictions = model.predict(X_test)
# Convert probabilities to binary predictions
binary_predictions = (predictions > 0.5).astype(int)

# Calculate metrics
accuracy = accuracy_score(y_test, binary_predictions)
precision = precision_score(y_test, binary_predictions)
recall = recall_score(y_test, binary_predictions)
f1 = f1_score(y_test, binary_predictions)

# Print the results
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, binary_predictions)
print("Confusion Matrix:\n", conf_matrix)


92/92 [==============================] - 18s 164ms/step
Accuracy: 0.7529853292391675
Precision: 0.7307692307692307
Recall: 0.8087837837837838
F1 Score: 0.7677998717126363
Confusion Matrix:
 [[1010  441]
 [ 283 1197]]


In [24]:
def build_model(hp):
    model = Sequential()
    model.add(Embedding(input_dim=vocab_size, output_dim=hp.Int('embedding_dim', min_value=32, max_value=512, step=32), input_length=43))
    model.add(Bidirectional(LSTM(units=hp.Int('units', min_value=32, max_value=512, step=32), dropout=hp.Float('dropout', 0, 0.5, step=0.1))))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=Adam(hp.Float('learning_rate', 1e-4, 1e-2, sampling='LOG')),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=3,
    directory='results',
    project_name='RNN_hlt')

tuner.search(X_train, y_train, epochs=5, validation_split=0.2)

Trial 10 Complete [00h 01m 24s]
val_accuracy: 0.7997075915336609

Best val_accuracy So Far: 0.8011695742607117
Total elapsed time: 00h 15m 28s


In [25]:
best_model = tuner.get_best_models(num_models=1)[0]

In [26]:
best_model

In [28]:
trials = tuner.oracle.get_best_trials()
for trial in trials:
    print('Trial ID:', trial.trial_id)
    print('Score:', trial.score)
    print('Hyperparameters:', trial.hyperparameters.values)


Trial ID: 06
Score: 0.8011695742607117
Hyperparameters: {'embedding_dim': 224, 'units': 192, 'dropout': 0.30000000000000004, 'learning_rate': 0.003814120220610665}


In [29]:
# Assuming X_test is prepared and ready
predictions = best_model.predict(X_test)
# Convert probabilities to binary predictions
binary_predictions = (predictions > 0.5).astype(int)

# Calculate metrics
accuracy = accuracy_score(y_test, binary_predictions)
precision = precision_score(y_test, binary_predictions)
recall = recall_score(y_test, binary_predictions)
f1 = f1_score(y_test, binary_predictions)

# Print the results
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, binary_predictions)
print("Confusion Matrix:\n", conf_matrix)


92/92 [==============================] - 2s 6ms/step
Accuracy: 0.7809621289662231
Precision: 0.7648546144121365
Recall: 0.8175675675675675
F1 Score: 0.7903331156107121
Confusion Matrix:
 [[1079  372]
 [ 270 1210]]


In [32]:
from tensorflow.keras.layers import Layer
import tensorflow as tf

class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1),
                                 initializer="random_normal")
        self.b = self.add_weight(name="att_bias", shape=(input_shape[1], 1),
                                 initializer="zeros")
        super(AttentionLayer, self).build(input_shape)

    def call(self, inputs):
        # Applying a simple attention mechanism
        e = tf.nn.tanh(tf.tensordot(inputs, self.W, axes=1) + self.b)
        a = tf.nn.softmax(e, axis=1)
        output = inputs * a
        return tf.reduce_sum(output, axis=1)

    def get_config(self):
        return super(AttentionLayer, self).get_config()


In [48]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional

# Model parameters
embedding_dim = 32 # Dimension of embedding space
lstm_units = 32     # Number of units in LSTM layer

# Define the model
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=43),
    Bidirectional(LSTM(lstm_units, dropout=0.3, return_sequences=True)),  # Ensure return_sequences is True for attention
    AttentionLayer(),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()


Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_10 (Embedding)    (None, 43, 32)            191456    
                                                                 
 bidirectional_10 (Bidirect  (None, 43, 64)            16640     
 ional)                                                          
                                                                 
 attention_layer_8 (Attenti  (None, 64)                107       
 onLayer)                                                        
                                                                 
 dense_10 (Dense)            (None, 1)                 65        
                                                                 
Total params: 208268 (813.55 KB)
Trainable params: 208268 (813.55 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [49]:
model.fit(X_train, y_train, batch_size=32, epochs=10, validation_split=0.2)
model.evaluate(X_test, y_test)

Epoch 1/10
171/171 [==============================] - 14s 47ms/step - loss: 0.5982 - accuracy: 0.6626 - val_loss: 0.5107 - val_accuracy: 0.7522
Epoch 2/10
171/171 [==============================] - 4s 23ms/step - loss: 0.4731 - accuracy: 0.7829 - val_loss: 0.4793 - val_accuracy: 0.7763
Epoch 3/10
171/171 [==============================] - 2s 14ms/step - loss: 0.3776 - accuracy: 0.8459 - val_loss: 0.4592 - val_accuracy: 0.7917
Epoch 4/10
171/171 [==============================] - 2s 12ms/step - loss: 0.3193 - accuracy: 0.8761 - val_loss: 0.4887 - val_accuracy: 0.7917
Epoch 5/10
171/171 [==============================] - 2s 13ms/step - loss: 0.2806 - accuracy: 0.8934 - val_loss: 0.5073 - val_accuracy: 0.7858
Epoch 6/10
171/171 [==============================] - 2s 12ms/step - loss: 0.2478 - accuracy: 0.9088 - val_loss: 0.5251 - val_accuracy: 0.7895
Epoch 7/10
171/171 [==============================] - 3s 16ms/step - loss: 0.2330 - accuracy: 0.9143 - val_loss: 0.5635 - val_accuracy: 0.788

[0.6087993383407593, 0.7703855633735657]